<a href="https://colab.research.google.com/github/amirpaia/blenderbot/blob/main/french_wow_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
# mydrive_path = '/content/drive/MyDrive/colabs/blender-models/'
# !pip uninstall -q parlai
!pip install -q parlai
!pip install transformers

In [1]:
!wget http://parl.ai/downloads/wizard_of_wikipedia/wizard_of_wikipedia.tgz
!mkdir /usr/local/lib/python3.7/dist-packages/data
!mkdir /usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia
!cp wizard_of_wikipedia.tgz /usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia/
!tar -zxvf /usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia/wizard_of_wikipedia.tgz -C /usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia/

--2022-06-09 11:50:51--  http://parl.ai/downloads/wizard_of_wikipedia/wizard_of_wikipedia.tgz
Resolving parl.ai (parl.ai)... 108.156.83.34, 108.156.83.128, 108.156.83.113, ...
Connecting to parl.ai (parl.ai)|108.156.83.34|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://parl.ai/downloads/wizard_of_wikipedia/wizard_of_wikipedia.tgz [following]
--2022-06-09 11:50:51--  https://parl.ai/downloads/wizard_of_wikipedia/wizard_of_wikipedia.tgz
Connecting to parl.ai (parl.ai)|108.156.83.34|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://dl.fbaipublicfiles.com/parlai/wizard_of_wikipedia/wizard_of_wikipedia.tgz [following]
--2022-06-09 11:50:51--  https://dl.fbaipublicfiles.com/parlai/wizard_of_wikipedia/wizard_of_wikipedia.tgz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load the dataset

In [ ]:
!parlai display_data --task wizard_of_wikipedia

[building data: /usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia]
19:43:42 | Downloading http://parl.ai/downloads/wizard_of_wikipedia/wizard_of_wikipedia.tgz to /usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia/wizard_of_wikipedia.tgz
19:44:27 | loading /usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia/train.json
19:44:37 | Some data not being used. If you are not trying to reproduce the previous results, it is recommended that you run with the flag --add-missing-turns train or --add-missing-turns all.
- - - NEW EPISODE: wizard_of_wikipedia - - -
Science fiction
   I think science fiction is an amazing genre for anything. Future science, technology, time travel, FTL travel, they're all such interesting concepts.
I'm a huge fan of science fiction myself! 
   Awesome! I really love how sci-fi storytellers focus on political/social/philosophical issues that would still be around even in the future. Makes them relatable.
I agree. One of my favorit

In [3]:
lines = []
with open("/usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia/data.json") as f:
    lines = f.readlines()
lines[0]

In [ ]:
import json
obj = json.loads(lines[0])

In [ ]:
print(len(obj))
obj[0]

In [ ]:
print(len(obj))
len([a['text'] for b in obj for a in b['dialog']])

# Translation

In [ ]:
!pip install translate-api

In [ ]:
import translators as ts
# print(ts._google.language_map)
import time
import os.path


def intTryParse(value):
    value = value.replace(" ","")
    value = value.replace("\u200b","")
    try:
        return int(value), True
    except ValueError:
        return value, False

lastest_dialog = -1
is_parsed = True
output_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_wow.txt"

if os.path.exists(output_path):
    with open(output_path) as f:
        lines = f.readlines()

    for i in range(1,len(lines)+1):
        line = lines[-i]
        if "<span class = 'notranslate'>" in line:
            lastest_dialog, is_parsed = intTryParse(line[line.find(">")+1:line.find("<",1)])
            if is_parsed: break
print(lastest_dialog)

tic = time.perf_counter()

if is_parsed or lastest_dialog == -1:
    text = ""
    # tic = time.perf_counter()
    for index, item in enumerate(obj):
        if index > lastest_dialog:
            item_text = '\n'.join( [a['text'] for a in item['dialog']])
            # print(index, "\n", item_text)

            text += f"\n<span class='notranslate'>{index}<span>\n{item_text}" 
            if (index + 1) % 3 == 0:
                translated = ts.google(text, from_language='en', to_language='fr')
                # print(translated)
                text = ""
                with open(output_path, "a") as f:
                    f.writelines("\n" + translated)
            
            if (index + 1) % 100 == 0:
                toc = time.perf_counter()
                print(f"{index + 1}\t{toc - tic:0.4f}\t", time.strftime("%H:%M:%S", time.gmtime()))
                tic = toc


    # print(f"Downloaded the tutorial in {toc - tic:0.4f} seconds")
else:
    print("Error")

Using United States server backend.


22310


In [ ]:
# obj[308]
[a['text'] for a in obj[308]['dialog']]

['Do you think Computer science or the study of the theory, experimentation, and engineering is the most useful degree? ',
 'I definitely think it has a significant place in the modern age',
 'I think the study of  automating algorithmic processes is going to be to the next generation  what the business degree was to mine.',
 'Right, there will definitely be a great need for computer scientists',
 'I can only hope computer scientist also retain so of the common sense that you need in the real world. ',
 'Right, I guess the business students will still be needed in that case',
 'I guess we will always need people with finance, accounting, marketing or strategy degrees.',
 'Right, I wonder if we will see people moving away from those degrees towards computer science',
 'I think we all ready have, well Computer science and other degrees that focus on challenges in implementing computation.',
 'That makes sense!']

Reading the generated file

In [ ]:
output_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_wow.txt"
with open(output_path) as f:
    fr_lines = f.readlines()

In [ ]:
len(fr_lines)

224310

# Correct translation mistakes

In [ ]:
output_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_wow_1.txt"

dialog = []
all_dialogs = []

all_dialogs_in_parlai_format = []
number_of_odd_dialogs = 0
number_of_dialog = 0

flag = False
dialog_index =0
for index, line in enumerate(fr_lines[1:]):
    if "<span" in line:
        if not flag:
            all_dialogs.append(dialog)
            dialog = []
        else:
            all_dialogs.append([])
            dialog = []
            flag = False
        
        dialog_index +=1
        if "<span class='notranslate'>" in line: 
            flag = True
    else:
        dialog.append(line)
all_dialogs.append(dialog)

# print(len([1 for a in all_dialogs if a == []]))
print(len(all_dialogs))
text = ""

count = 0
previous_index = 1000
for index, dialog in enumerate(all_dialogs):
    if index >= previous_index:
        if dialog == []: 
            count +=1
            item_text = '\n'.join( [a['text'].replace("\n","") for a in obj[index]['dialog']])
            text += f"\n<span class='notranslate'> {index}</span>\n{item_text}" 
            translated = ts.google(text, from_language='en', to_language='fr')
            all_dialogs[index] = "\n" + translated
            print(count, index)
            text = ""
        else:
            item_text = '\n'.join([a.replace("\n","") for a in dialog])
            text += f"\n<span class = 'notranslate'> {index} </span>\n{item_text}" 
            all_dialogs[index] = text
            text = ""
        
        if index % 5 == 0:
            with open(output_path, "a") as f:
                f.writelines(all_dialogs[previous_index:index])
                previous_index = index

# Check final version

In [ ]:
output_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_wow_1.txt"
with open(output_path) as f:
    fr_lines = f.readlines()
dialog_index = 0
for line in fr_lines:
    if "<span" in line:
        latest_dialog, is_parsed = intTryParse(line[line.find(">")+1:line.find("<",1)])
        if dialog_index != latest_dialog:
            print("break", dialog_index, latest_dialog)
            break
        dialog_index +=1
        # break
        # print(index, line.replace("\n",""), latest_dialog, number_of_dialog)
print("done!")

# Convert to ParlAI format

In [10]:
def transfer_list_of_turns_to_dialog(d):
    if len(d)%2 !=0: d = d[:-1]
    t = ""
    for i in range(0,len(d),2):
        u1 = d[i]
        u2 = d[i+1]

        if (i+2) != len(d):
            t += "text:"+u1+"\t"+"labels:"+u2+"\n"
        else:
            t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"
    return t

In [11]:
output_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_wow.txt"
with open(output_path) as f:
    fr_lines = f.readlines()
dialog_index = 0

dialog = []
all_dialogs = []
all_dialogs_in_parlai_format = []

number_of_odd_dialogs = 0
number_of_dialog = 0

for line in fr_lines[1:]:
    if "<span" in line:
        # latest_dialog, is_parsed = intTryParse(line[line.find(">")+1:line.find("<",1)])
        if len(dialog) % 2 ==1: number_of_odd_dialogs +=1
        number_of_dialog +=1
        all_dialogs_in_parlai_format.append(transfer_list_of_turns_to_dialog(dialog))
        dialog = []
    else:
        dialog.append(line)

print("done!", len(all_dialogs_in_parlai_format), number_of_dialog, number_of_odd_dialogs)

done! 22310 22310 1029


In [12]:
import pandas as pd
import numpy as np

data_path = '/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_wizard_of_wikipedia/'
# with open(data_path + "train.txt", "w") as f:
#     f.writelines(dialogs_in_parlai_forma

df = pd.DataFrame ([a[0] for a in all_dialogs_in_parlai_format], columns = ['dialog'])
train, valid, test = np.split(df.sample(frac=1, random_state=42), 
                                 [int(.8*len(df)), 
                                  int(.9*len(df))])
print(f"train set: {len(train)}, validation set: {len(valid)},test set: {len(test)}")

with open(f"{data_path}/train.txt","w") as f:    
    f.write(''.join(a[0] for a in train.values))

with open(f"{data_path}/valid.txt","w") as f:
    f.write(''.join(a[0] for a in valid.values))

with open(f"{data_path}/test.txt","w") as f:
    f.write(''.join(a[0] for a in test.values))
print('done!')

train set: 17848, validation set: 2231,test set: 2231
done!
